In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy as sci
import rpy2
print(rpy2.__version__)
import gseapy as gp
from xlrd import XLRDError
import xlsxwriter
from gprofiler import gprofiler
import rpy2.robjects as ro
import os

/home/glennrd/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


3.5.16


## Differential Gene Expression Analysis

In [2]:
adata_crypts = sc.read_h5ad('/home/glennrd/Documents/Research_Project/RNA-seq_Analysis/python/crypt/crypt_enriched.h5ad')
adata_villi = sc.read_h5ad('/home/glennrd/Documents/Research_Project/RNA-seq_Analysis/python/villi/villus_enriched.h5ad')

In [3]:
# Create batch annotation for limma (Crypts)
adata_crypts.obs['Batch'] = ['batch'+x.split('_')[1] for x in adata_crypts.obs['Sample']]
adata_crypts.obs['Batch'] = adata_crypts.obs['Batch'].astype('category')

In [4]:
# Create batch annotation for limma (villi)
adata_villi.obs['Batch'] = adata_villi.obs['diet'].map({'CD': 'batch1', 'HFD': 'batch2'})
adata_villi.obs['Batch'] = adata_villi.obs['Batch'].astype('category')
adata_villi.obs.rename(columns={'diet': 'Diet'}, inplace=True)

In [5]:
def abs_log2fc(adata,grouptype,group2,group1):
    # calculate the absolute log2-foldchange as log2(b/a) = (ln(b)-ln(a))/ln(2)
    # data is ln(counts+1), neglect the +1
    obs_df=pd.DataFrame(adata.X.toarray(),columns=adata.var_names)
    obs_df[grouptype]=adata.obs[grouptype].values    
    grouped_df=obs_df.groupby([grouptype])
    tot=grouped_df[adata.var_names].apply(lambda x: np.mean(x, axis=0))
    print("Group2:", group2, "in tot:", group2 in tot.index)
    print("Group1:", group1, "in tot:", group1 in tot.index)
    lfc= (tot.loc[group2] - tot.loc[group1])/np.log(2)
    return lfc

In [18]:
def run_limma_counts(adata, condobs, batchobs, countsobs, coef,ref):

    """
    This function performs differential gene expression analysis using the limma and edgeR packages from R.
    It takes an AnnData object, column names for condition, batch, counts, a coefficient for the limma model, 
    and a reference level for the condition factor. It returns a DataFrame containing the results of the 
    differential expression analysis.
    """

    import rpy2.robjects as robjects
    import rpy2.robjects.numpy2ri
    from rpy2.robjects import pandas2ri
    from rpy2.robjects.packages import importr
    pandas2ri.activate()
    
    # prepare data for R
    data=pd.DataFrame(adata.X.toarray(), columns=adata.var_names)
    cond=adata.obs[condobs]
    batch=adata.obs[batchobs]
    counts=adata.obs[countsobs]

    # load R packages and data
    R=robjects.r
    print('Loading edgeR package')
    R('library(edgeR)')
    print('Loading limma package')
    R('library(limma)')
    print('Assigning data ⌛')
    R.assign('data',data.T)
    #print(dgelist)
    print('Assigning condition')
    R.assign('cond', cond)
    print('Assigning batch')
    R.assign('batch', batch)
    print('Assigning counts')
    R.assign('counts', counts)
    R('counts<-scale(counts)')

    # delete for memory
    del data
    del cond    
    del batch
    
    # format data and create dge object 
    print('Formatting data')
    R('cond <- as.factor(cond)')
    print('Releveling condition')
    R('cond <- relevel(cond,'''+ref+''')''')
    print('Creating DGEList')
    R('dge <- edgeR::DGEList(data,group=cond,sample=data.frame(batch=batch,counts=counts))')
    print('Deleting data')
    R('rm(data)')
    print('Creating EList')
    R('y <- new("EList")')
    print('Assigning DGEList')
    R('y$E <- dge')
    print('Deleting DGEList')
    R('rm(dge)')
    
    # design matrix for testing
    print('Creating design matrix')
    R('design <- model.matrix(~cond+batch+counts)')
    
    # run limma
    print('Running Limma lmFit ⌛')
    R('fit <- limma::lmFit(y, design = design)')
    R('rm(y)')
    print('Running Limma eBayes ⌛')
    R('fit <-  limma::eBayes(fit, trend = TRUE, robust = TRUE,)')
    
    # get results
    print('Getting results')
    strg='"BH"'
    tt = R('''limma::topTable(fit,coef='''+coef+''',number = Inf,adjust.method='''+strg+''')''')
    return tt

In [7]:
def filter_amb(adata):
    # ambient genes for filtering, see processing notebook
    ambient_genes=['Itln1','Spink4','Zg16','Lyz1','Defa21','Gm14851','Defa22','Gm15308','Gm15284',
                   'Defa20','Gm15308','Gm14850','Gm7861','Defa17','AY761184', 'Ang4','Agr2','Clps','Tff3','Defa24','Fcgbp']
    ix_amb_genes = np.in1d(adata.var_names,ambient_genes,invert=True)
    return (ix_amb_genes)

In [8]:
def filter_genes(adata_filt, obs_name, group1, group2):
    # Filter genes expressed in at least 10% of cells in either group
    ix_group1 = np.isin(adata_filt.obs[obs_name], group1)
    adata_sub_group1 = adata_filt[ix_group1].copy()
    filter_1 = sc.pp.filter_genes(adata_sub_group1.X, min_cells=int(adata_sub_group1.n_obs * 0.1), inplace=False)[0]
    del adata_sub_group1

    ix_group2 = np.isin(adata_filt.obs[obs_name], group2)
    adata_sub_group2 = adata_filt[ix_group2].copy()
    filter_2 = sc.pp.filter_genes(adata_sub_group2.X, min_cells=int(adata_sub_group2.n_obs * 0.1), inplace=False)[0]
    del adata_sub_group2
    
    ix_genes=[a or b for a, b in zip(filter_1,filter_2)]
    
    adata_filt = adata_filt[:,np.array(ix_genes)].copy()
    
    return adata_filt

In [9]:
def write_to_csv(df, filename, group, prefix):
    if len(group)>17:
            group_short=group[0:17]
            group_short=group_short.replace('/', '_')
            group_short=group_short.replace(' ', '_')
            
            df.to_csv(filename + "_" + group_short + prefix + ".csv")
    else:
            group=group.replace('/', '_')
            group=group.replace(' ', '_')

            df.to_csv(filename + "_" + group + prefix + ".csv")

In [66]:
cells_crypts = adata_crypts.obs['major_cell_types'].unique()
print('List of unique cells in crypt data:')
for cell in cells_crypts:
    print(cell)

print('\n')

cells_villi = adata_villi.obs['major_cell_types'].unique()
print('List of unique cells in villi data:')
for cell in cells_villi:
    print(cell)

List of unique cells in crypt data:
Enterocyte progenitor
Goblet cell
EEC
Goblet progenitor
ISC
not annotated
Tuft progenitor
Enterocyte
Tuft cell
EE progenitor
Unkown progenitor
Paneth cell
Paneth progenitor


List of unique cells in crypt data:
Goblet cell
Tuft cell
Enterocyte progenitor
Enterocyte
EEC
ISC


In [19]:
# Function to perform differential expression analysis
def differential_expression_analysis(list_cells, adata, dataset, group):
    # Change working directory to the appropriate path
    os.chdir(f'/home/glennrd/Documents/Research_Project/RNA-seq_Analysis/python/{dataset}/differential_expression')

    for cell in list_cells:
        # Start differential testing for the cell type
        print(f'Performing differential expression analysis of {cell} cells ⌛')

        # Select the cell type
        ix_cells = np.isin(adata.obs[group], [cell])

        # Create a subset of the data containing only those cell types
        adata_filt = adata[ix_cells, :].copy()
        print(f'Subsetting {np.sum(ix_cells)} cells')

        # Filter out ambient genes
        print('Removing ambient genes ⌛')
        ix_amb = filter_amb(adata) 
        adata_filt = adata_filt[:, ix_amb].copy()

        # Filter out genes that are expressed in less than 10% of cells in either group (CD or HFD)
        print('Removing genes expressed in less than 10% of cells ⌛')
        adata_filt = filter_genes(adata_filt, 'Diet', 'CD', 'HFD')

        # Run differential expression analysis using limma
        print('Running Limma ⌛')

        try:
            # Run limma counts with 'diet' as the condition, 'batch' as the batch, 'n_genes' as the counts, 
            # 'condHFD' as the coefficient for the limma model, and 'CD' as the reference level for the condition factor
            x = run_limma_counts(adata_filt, 'Diet', 'Batch', 'n_genes', '"condHFD"', '"CD"')

        except rpy2.rinterface_lib.embedded.RRuntimeError:
            # If no significant genes are found, print a message and continue
            print('No significant genes found..')
            continue

        print('Data stored in variable x')

        print('Calculating absolute lfc')

        base_filename = 'diff_exp_CD_vs_HFD'

        # Compute absolute log2 fold change
        lfc = abs_log2fc(adata_filt, 'Diet', 'HFD', 'CD')
        x = ro.conversion.rpy2py(x)
        x = x.loc[lfc.index]
        x['abs.log2FC'] = lfc.values
        del lfc

        # Write to csv
        print(f'Writing to {base_filename}_{cell} ⌛')
        write_to_csv(pd.DataFrame(x), base_filename, cell, '')
        print('Done ✅')

In [68]:
# Define variables for adata_villi
list_cells_villi = cells_villi
dataset_villi = 'villi'
group_villi = 'major_cell_types'

# Define variables for adata_crypts
list_cells_crypts = cells_crypts
dataset_crypts = 'crypt'
group_crypts = 'major_cell_types'

# Run differential expression analysis for adata_villi
differential_expression_analysis(list_cells_villi, adata_villi, dataset_villi, group_villi)

# Run differential expression analysis for adata_crypts
differential_expression_analysis(list_cells_crypts, adata_crypts, dataset_crypts, group_crypts)


R[write to console]: Loading required package: limma



Performing differential expression analysis of Goblet cell cells ⌛
Subsetting 1001 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Assigning counts
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Coefficients not estimable: batchbatch2 


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: Partial NA coefficients for 7211 probe(s) 

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In out$var.prior[is.na(out$var.prior)] <- 1/out$s2.prior :
R[write to console]: 
 
R[write to console]:  number of items to replace is not a multiple of replacement length

R[write to console]: 2: 
R[write to console]: In .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim,  :
R[write to console]: 
 
R[write to console]:  Estimation of var.prior failed - set to default value



Running Limma eBayes ⌛
Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_Goblet cell ⌛
Done ✅
Performing differential expression analysis of Tuft cell cells ⌛
Subsetting 1267 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Assigning counts
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Coefficients not estimable: batchbatch2 


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: Partial NA coefficients for 8352 probe(s) 

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In out$var.prior[is.na(out$var.prior)] <- 1/out$s2.prior :
R[write to console]: 
 
R[write to console]:  number of items to replace is not a multiple of replacement length

R[write to console]: 2: 
R[write to console]: In .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim,  :
R[write to console]: 
 
R[write to console]:  Estimation of var.prior failed - set to default value



...

Running Limma eBayes ⌛
Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_Enterocyte ⌛
Done ✅
Performing differential expression analysis of EEC cells ⌛
Subsetting 331 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Assigning counts
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Coefficients not estimable: batchbatch2 


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: Partial NA coefficients for 8828 probe(s) 

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In out$var.prior[is.na(out$var.prior)] <- 1/out$s2.prior :
R[write to console]: 
 
R[write to console]:  number of items to replace is not a multiple of replacement length

R[write to console]: 2: 
R[write to console]: In .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim,  :
R[write to console]: 
 
R[write to console]:  Estimation of var.prior failed - set to default value



Running Limma eBayes ⌛
Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_EEC ⌛
Done ✅
Performing differential expression analysis of ISC cells ⌛
Subsetting 115 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Assigning counts
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Coefficients not estimable: batchbatch2 


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: Partial NA coefficients for 11416 probe(s) 

R[write to console]: In addition: 
R[write to console]: Warning messages:

R[write to console]: 1: 
R[write to console]: In out$var.prior[is.na(out$var.prior)] <- 1/out$s2.prior :
R[write to console]: 
 
R[write to console]:  number of items to replace is not a multiple of replacement length

R[write to console]: 2: 
R[write to console]: In .ebayes(fit = fit, proportion = proportion, stdev.coef.lim = stdev.coef.lim,  :
R[write to console]: 
 
R[write to console]:  Estimation of var.prior failed - set to default value



Running Limma eBayes ⌛
Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_ISC ⌛
Done ✅
Performing differential expression analysis of Enterocyte progenitor cells ⌛
Subsetting 8325 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Assigning counts
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Running Limma eBayes ⌛
Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_Enterocyte progenitor ⌛
Done ✅
Performing differential expression analysis of Goblet cell cells ⌛
Subsetting 4539 cells
Removing ambient genes ⌛
Removing genes expressed in less t

In [23]:
adata_crypts_pb = sc.read_h5ad('/home/glennrd/Documents/Research_Project/RNA-seq_Analysis/python/crypt/crypt_enriched_pseudobulk.h5ad')

# Create batch annotation for limma (Crypts)
adata_crypts_pb.obs['Batch'] = ['batch'+x.split('_')[1] for x in adata_crypts_pb.obs['Sample']]
adata_crypts_pb.obs['Batch'] = adata_crypts_pb.obs['Batch'].astype('category')

cells_crypts_pb = adata_crypts_pb.obs['major_cell_types'].unique()
print('List of unique cells in crypt data:')
for cell in cells_crypts_pb:
    print(cell)

# Define variables for adata_crypts_pb
list_cells_crypts_pb = cells_crypts_pb
dataset_crypts_pb = 'crypt_pb'
group_crypts_pb = 'major_cell_types'

# Run differential expression analysis for adata_villi
differential_expression_analysis(list_cells_crypts_pb, adata_crypts_pb, dataset_crypts_pb, group_crypts_pb)

List of unique cells in crypt data:
Intestinal epithelium
Performing differential expression analysis of Intestinal epithelium cells ⌛
Subsetting 6 cells
Removing ambient genes ⌛
Removing genes expressed in less than 10% of cells ⌛
Running Limma ⌛
Loading edgeR package
Loading limma package
Assigning data ⌛
Assigning condition
Assigning batch
Formatting data
Releveling condition
Creating DGEList
Deleting data
Creating EList
Assigning DGEList
Deleting DGEList
Creating design matrix
Running Limma lmFit ⌛
Running Limma eBayes ⌛


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: 44 very small variances detected, have been offset away from zero 



Getting results
Data stored in variable x
Calculating absolute lfc
Group2: HFD in tot: True
Group1: CD in tot: True
Writing to diff_exp_CD_vs_HFD_Intestinal epithelium ⌛
Done ✅
